<a href="https://colab.research.google.com/github/likeaAI/ML_3sdaq/blob/master/demo_styleclassfication_cloths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from glob import glob

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="V1xrnYqaT6VW3skv3nh8")
project = rf.workspace("example01").project("test03")
dataset = project.version(3).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 5.0 MB/s 
     |████████████████████████████████| 178 kB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 55.6 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 138 kB 62.1 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=95ec7223f571199f5f0e22700af8b5a62f30a29d19cd10664b0cb3d5bf724732
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=515d9cf1c98a2825c71fc0bbf15907eb0373db69652a4734a5bacbc5e42edcae
  Stored in directory: /root/.cache/pip/

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to test03-3 in yolov5pytorch:: 100%|██████████| 2297/2297 [00:02<00:00, 938.77it/s] 


In [ ]:
!nvidia-smi

Thu Jun  2 07:05:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# yolov5 다운로드 
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 12169, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 12169 (delta 0), reused 2 (delta 0), pack-reused 12167
Receiving objects: 100% (12169/12169), 11.95 MiB | 13.20 MiB/s, done.
Resolving deltas: 100% (8427/8427), done.


In [ ]:
# 이미지 list 
train_img_list = glob('/content/test03-3/train/images/*.jpg')
val_img_list = glob('/content/test03-3/valid/images/*.jpg')
test_img_list = glob('/content/test03-3/test/images/*.jpg')

# label list 
train_label_list = glob('/content/test03-3/train/labels/*.txt')
val_label_list = glob('/content/test03-3/valid/labels/*.txt')
test_label_list = glob('/content/test03-3/test/labels/*.txt')

print(len(train_img_list) , len(val_img_list) , len(test_img_list))
print(len(train_label_list) , len(val_label_list) , len(test_label_list))

1089 55 0
1089 55 0


In [ ]:
with open('/content/test03-3/train.txt' , 'w') as f : 
    f.write('\n'.join(train_img_list))
with open('/content/test03-3/val.txt' , 'w') as f : 
    f.write('\n'.join(val_img_list))
with open('/content/test03-3/test.txt' , 'w') as f : 
    f.write('\n'.join(test_img_list))

with open('/content/test03-3/train.txt' , 'a') as f : 
    f.write('\n'.join(train_label_list))
with open('/content/test03-3/val.txt' , 'a') as f : 
    f.write('\n'.join(val_label_list))
with open('/content/test03-3/test.txt' , 'a') as f : 
    f.write('\n'.join(test_label_list))

In [ ]:
import yaml

with open('/content/test03-3/data.yaml' , 'r') as f : 
    data = yaml.safe_load(f) # yaml.load() x => yaml.safe_load(f)


data['train'] = '/content/test03-3/train.txt'   
data['val'] = '/content/test03-3/val.txt'
data['test'] = '/content/test03-3/test.txt'

   

with open ('/content/test03-3/data.yaml' , 'w') as f : 
    yaml.dump(data, f )

print(data)

{'names': ['bottom', 'bottom_short', 'top'], 'nc': 3, 'train': '/content/test03-3/train.txt', 'val': '/content/test03-3/val.txt', 'test': '/content/test03-3/test.txt'}


In [ ]:
%cat /content/test03-3/data.yaml

names:
- bottom
- bottom_short
- top
nc: 3
test: /content/test03-3/test.txt
train: /content/test03-3/train.txt
val: /content/test03-3/val.txt


In [ ]:
# 라벨인식을 못하는 문제 발생함 !!!! 확인할것. 
# 이미지와 라벨파일을 같은 txt에 추가하여 해결함 
# yolov5s 가장 가벼운 모델 , yolov5l은 무거운 모델 성능이 조금 더 좋음 

%cd /content/yolov5/

!python train.py --img 416 --batch 50 --epochs 35 \
--data /content/test03-3/data.yaml \
--cfg ./models/yolov5l.yaml \
--weights yolov5l.pt \
--name test03_result

/content/yolov5
train: weights=yolov5l.pt, cfg=./models/yolov5l.yaml, data=/content/test03-3/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=35, batch_size=50, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=test03_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-243-g7cef03d Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0

In [ ]:
# 추론하기 
from IPython.display import Image
import os


# /content/test03-2/test/images/ 이미지파일 판정

!python detect.py  --source /content/test03-3/test --weights /content/yolov5/runs/train/test02_result3/weights/best.pt \
 --img 416 --conf 0.5 --source /content/test03-1/test/images/

detect: weights=['/content/yolov5/runs/train/test02_result3/weights/best.pt'], source=/content/test03-1/test/images/, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.65, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-243-g7cef03d Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients
image 1/10 /content/test03-1/test/images/test62_jpg.rf.86c1910bf5f74be641c4425a1b634a71.jpg: 416x416 1 bottom, Done. (0.007s)
image 2/10 /content/test03-1/test/images/test63_jpg.rf.07b9b04ac10bccfe7c137829ce166c83.jpg: 416x416 1 bottom, Done. (0.007s)
image 3/10 /content/test03-1/test/images/test64_jpg.rf.63a4f7127eb451